# 01 [worked] but mismatch residue numbering with given pdb

In [1]:
# -------------------------------
# Step 0: Install Dependencies
# -------------------------------
!pip install biopython pdb-tools py3Dmol

# -------------------------------
# Step 1: Download and Install Modeller
# -------------------------------
!wget -q https://salilab.org/modeller/10.4/modeller-10.4.tar.gz
!tar -zxf modeller-10.4.tar.gz
!mkdir -p /content/compiled/MODELLER

%cd modeller-10.4
with open('modeller_config', 'w') as f:
    f.write("2\n")
    f.write("/content/compiled/MODELLER\n")
    f.write("MODELIRANJE\n")  # <-- Replace with your license key if needed
!./Install < modeller_config

# Link mod10.4 to PATH
!ln -sf /content/compiled/MODELLER/bin/mod10.4 /usr/bin/
!mod10.4 | awk 'NR==1{if($1=="usage:") print "✅ Modeller successfully installed"; else print "❌ Installation failed"}'
%cd /content

# -------------------------------
# Step 2: Download Input Files
# -------------------------------
# In this example, we use PDB ID 4bgq and UniProt ID O76039.
# (We will use the PDB's SEQRES information for numbering.)
pdb_id = "4bgq"
uniprot_id = "O76039"

!wget -q https://files.rcsb.org/download/{pdb_id}.pdb
!wget -q https://www.uniprot.org/uniprot/{uniprot_id}.fasta -O {uniprot_id}.fasta

# Create working directory and move files
!mkdir -p /content/4bgq_fix
!mv {pdb_id}.pdb {uniprot_id}.fasta /content/4bgq_fix/
%cd /content/4bgq_fix

# Save a copy of the original PDB (with SEQRES records) for sequence extraction.
!cp {pdb_id}.pdb {pdb_id}_orig.pdb

# -------------------------------
# Step 3: Clean the PDB (Keep ATOM records only)
# -------------------------------
from Bio.PDB import PDBParser, PDBIO, Select

class StandardResidueSelect(Select):
    def accept_residue(self, residue):
        # Accept only standard amino acid residues (skip hetero atoms)
        return residue.id[0] == ' '

parser = PDBParser(QUIET=True)
structure = parser.get_structure(pdb_id, f"{pdb_id}.pdb")
io = PDBIO()
io.set_structure(structure)
io.save("4bgq_clean.pdb", select=StandardResidueSelect())
print("✅ Cleaned PDB saved as 4bgq_clean.pdb")

# For modeling, Modeller must see a file named exactly as the template in the PIR header.
# Replace the original PDB with the cleaned version.
!cp 4bgq_clean.pdb 4bgq.pdb

# -------------------------------
# Step 4: Extract Full Intended Sequence from SEQRES Records
# -------------------------------
from Bio.Data.IUPACData import protein_letters_3to1

def three_to_one(resname):
    # Convert three-letter code to one-letter code.
    return protein_letters_3to1.get(resname.capitalize(), 'X')

def get_seqres_sequence(pdb_file, chain_id="A"):
    # This function parses SEQRES lines from the original PDB file.
    full_seq = ""
    with open(pdb_file, "r") as f:
        for line in f:
            # SEQRES records should have the chain id in column 12 (index 11)
            if line.startswith("SEQRES") and line[11] == chain_id:
                parts = line.split()
                # Residue names start at the 5th field
                for res in parts[4:]:
                    full_seq += three_to_one(res)
    return full_seq

# Use the original file (with SEQRES) to extract the intended sequence.
full_seq = get_seqres_sequence(f"{pdb_id}_orig.pdb", chain_id="A")
full_length = len(full_seq)
print(f"✅ Full SEQRES sequence for chain A: {full_length} residues.")

# -------------------------------
# Step 5: Build Template Sequence from Observed ATOM Records
# -------------------------------
# Parse the cleaned PDB (which is now named 4bgq.pdb) to get observed residues.
from Bio.PDB import PDBParser

pdb_parser = PDBParser(QUIET=True)
structure = pdb_parser.get_structure(pdb_id, "4bgq.pdb")
chain = next(structure[0].get_chains())  # Assume chain A

# Build a dictionary mapping residue number -> one-letter residue (from ATOM records)
observed_dict = {}
for r in chain.get_residues():
    if r.id[0] == ' ':
        observed_dict[r.id[1]] = three_to_one(r.get_resname())

if observed_dict:
    observed_start = min(observed_dict.keys())
    observed_end = max(observed_dict.keys())
    print(f"✅ Observed ATOM records span from residue {observed_start} to {observed_end}")
else:
    raise Exception("No observed residues found in the cleaned PDB.")

# For each position from 1 to full_length (e.g. 302 if that’s the SEQRES length),
# use the observed residue if present, otherwise add a gap ("-")
template_seq = "".join([observed_dict.get(i, "-") for i in range(1, full_length+1)])
assert len(template_seq) == full_length, f"Template sequence length {len(template_seq)} != {full_length}"
print("✅ Template sequence constructed.")


# -------------------------------
# (Assuming Steps 0 to 5 have already run.)
# We now have:
# - observed_start (e.g., 9) and observed_end (e.g., 302)
# - full_length (304, from SEQRES)
# - template_seq built for full_length positions (with gaps for missing residues)
# - full_seq from SEQRES (304 residues)
# -------------------------------

# Adjusted Step 6: Write the PIR Alignment File with Correct Template Header
with open("alignment.ali", "w") as f:
    f.write(f""">P1;{pdb_id}
structureX:{pdb_id}:{observed_start}:A:{observed_end}:A::::
{template_seq}*
>P1;target
sequence:target:1:A:{full_length}:A::::
{full_seq}*
""")
print(f"✅ Alignment written with template range {observed_start}-{observed_end} and target range 1-{full_length}.")

# -------------------------------
# Step 7: Run Modeller to Rebuild Missing Residues
# -------------------------------
modeller_script = """
from modeller import *
from modeller.automodel import *

log.verbose()
env = environ()
env.io.hetatm = True
env.io.atom_files_directory = ['.']

a = automodel(env,
              alnfile='alignment.ali',
              knowns='4bgq',
              sequence='target',
              assess_methods=(assess.DOPE, assess.GA341))
a.starting_model = 1
a.ending_model = 1
a.make()
"""

with open("rebuild_missing_residues.py", "w") as f:
    f.write(modeller_script)

print("✅ Modeller script written. Running it...")
!mod10.4 rebuild_missing_residues.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
/content/modeller-10.4
Installation of MODELLER 10.4

This script will install MODELLER 10.4 into a specified directory
for which you have read/write permissions.

To accept the default answers indicated in [...], press <Enter> only.

------------------------------------------------------------------------

The currently supported architectures are as follows:

   1) Linux x86 PC (e.g. RedHat, SuSe).
   2) x86_64 (Opteron/EM64T) box (Linux).
   3) Alternative x86 Linux binary (e.g. for FreeBSD).
   4) Linux on 32-bit ARM (e.g. for Raspberry Pi).
   5) Linux on 64-bit ARM (e.g. for Raspberry Pi 3 or later).

Select the type of your computer from the list above [2]: 

Full directory name in which to install MODELLER 10.4
[/root/bin/modeller10.4]: 

License key, obtained from our academic license server at
https://salilab.org/modelle

# [FIXED] Step: Fix Residue Numbering in Modeled PDB

In [2]:
!pwd

/content/4bgq_fix


In [9]:
# -------------------------------
# Step 8: Renumber Modeled PDB Based on Alignment with Insertion Codes (Fixed)
# -------------------------------
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.PDB import PDBParser, PDBIO
from Bio.Data.IUPACData import protein_letters_3to1

def three_to_one(resname):
    """Convert a three-letter residue code to one-letter."""
    return protein_letters_3to1.get(resname.capitalize(), "X")

def get_chain_sequence(pdb_file, chain_id="A"):
    """
    Parse a PDB file and extract the 1-letter sequence and residue objects
    from the specified chain.
    Returns: (sequence_string, list_of_residues, structure, chain)
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("model", pdb_file)
    model = next(structure.get_models())
    chain = None
    for ch in model.get_chains():
        if ch.get_id() == chain_id:
            chain = ch
            break
    if chain is None:
        chain = next(model.get_chains())

    seq = ""
    residues = []
    for res in chain.get_residues():
        if res.id[0] == " ":  # standard residues only
            seq += three_to_one(res.get_resname())
            residues.append(res)
    return seq, residues, structure, chain

# Input files and desired output.
model_pdb_file = "target.B99990001.pdb"  # Modeled PDB from Modeller
output_pdb_file = "target_renumbered.pdb"

# 1. Extract the modeled PDB sequence and its residues.
modeled_seq, residues, structure, chain = get_chain_sequence(model_pdb_file, chain_id="A")
print("✅ Modeled PDB sequence (length {}):".format(len(modeled_seq)))
print(modeled_seq)

# 2. Define the intended (target) sequence (from UniProt aa_range header).
intended_seq = (
    "MKIPNIGNVMNKFEILGVVGEGAYGVVLKCRHKETHEIVAIKKFKDSEENEEVKETTLRELKMLRTLKQENIVELKEAFRRRGKLYLVFEYVEKNMLELLEEMPNGVPPEKVKSYIYQLIKAIHWCHKNDIVHRDIKPENLL"
    "ISHNDVLKLCDFGFARNLSEGNNANYTEYVATRWYRSPELLLGAPYGKSVDMWSVGCILGELSDGQPLFPGESEIDQLFTIQKVLGPLPSEQMKLFYSNPRFHGLRFPAVNHPQSLERRYLGILNSVLLDLMKNLLKLDPADRYLTEQCLNHPTFQTQRLL"
)
print("\n✅ Intended UniProt sequence (length {}):".format(len(intended_seq)))
print(intended_seq)

# 3. Align modeled sequence vs. intended sequence.
alignments = pairwise2.align.globalms(modeled_seq, intended_seq, 2, -1, -0.5, -0.1)
best_aln = alignments[0]
print("\n🔍 Best alignment:")
print(format_alignment(*best_aln))

# 4. Build mapping from modeled residue indices (0-indexed) to target numbering.
#    We'll output each mapping as (target_number, insertion_code) where insertion_code will be a single character.
aligned_modeled = best_aln.seqA
aligned_intended = best_aln.seqB

map_model_to_target = {}  # key: modeled residue list index -> (target_num, insertion_code)
model_index = 0  # index into modeled residues list
target_index = 0  # intended sequence residue counter (for numbering)
insertion_counter = None  # tracks insertion letters during an insertion block

for pos, (char_model, char_intended) in enumerate(zip(aligned_modeled, aligned_intended)):
    if char_model != '-' and char_intended != '-':
        insertion_counter = None
        # Use a single space for no insertion.
        map_model_to_target[model_index] = (target_index + 1, " ")
        model_index += 1
        target_index += 1
    elif char_model != '-' and char_intended == '-':
        # Modeled extra relative to intended.
        if model_index == 0:
            print(f"⚠️ Extra residue {model_index} ({residues[model_index].get_resname()}) at beginning: will be removed.")
            map_model_to_target[model_index] = None  # mark for removal.
            model_index += 1
        else:
            if insertion_counter is None:
                insertion_counter = 0
            ins_code = chr(ord('A') + insertion_counter)
            map_model_to_target[model_index] = (target_index, ins_code)
            insertion_counter += 1
            model_index += 1
    elif char_model == '-' and char_intended != '-':
        insertion_counter = None
        target_index += 1
    # If both are '-', do nothing.

# 5. Rebuild a new chain based on the mapping.
from Bio.PDB.Chain import Chain
new_chain = Chain(chain.get_id())
new_residues = []
for idx, res in enumerate(residues):
    mapping = map_model_to_target.get(idx)
    if mapping is None:
        continue  # Skip residue (e.g., the extra beginning residue).
    target_num, ins_code = mapping
    # Ensure that the insertion code is exactly one character. If ins_code is empty, use a space.
    if not ins_code:
        ins_code = " "
    new_id = (" ", target_num, ins_code)
    res.id = new_id
    new_residues.append(res)
    new_chain.add(res)

print("\n✅ New chain constructed: {} residues retained.".format(len(new_residues)))

# 6. Build a new structure containing the new chain.
from Bio.PDB.Model import Model
from Bio.PDB.Structure import Structure

new_model = Model(0)
new_model.add(new_chain)
new_structure = Structure("renum")
new_structure.add(new_model)

# 7. Write out the renumbered PDB.
io = PDBIO()
io.set_structure(new_structure)
io.save(output_pdb_file)
print("\n✅ Final renumbered PDB saved as {}".format(output_pdb_file))


✅ Modeled PDB sequence (length 304):
SMKIPNIGNVMNKFEILGVVGEGAYGVVLKCRHKETHEIVAIKKFKDSEENEEVKETTLRELKMLRTLKQENIVELKEAFRRRGKLYLVFEYVEKNMLELLEEMPNGVPPEKVKSYIYQLIKAIHWCHKNDIVHRDIKPENLLISHNDVLKLCDFGFARNLSEGNNANYDEEVATRWYRSPELLLGAPYGKSVDMWSVGCILGELSDGQPLFPGESEIDQLFTIQKVLGPLPSEQMKLFYSNPRFHGLRFPAVNHPQSLERRYLGILNSVLLDLMKNLLKLDPADRYLTEQCLNHPTFQTQRLL

✅ Intended UniProt sequence (length 303):
MKIPNIGNVMNKFEILGVVGEGAYGVVLKCRHKETHEIVAIKKFKDSEENEEVKETTLRELKMLRTLKQENIVELKEAFRRRGKLYLVFEYVEKNMLELLEEMPNGVPPEKVKSYIYQLIKAIHWCHKNDIVHRDIKPENLLISHNDVLKLCDFGFARNLSEGNNANYTEYVATRWYRSPELLLGAPYGKSVDMWSVGCILGELSDGQPLFPGESEIDQLFTIQKVLGPLPSEQMKLFYSNPRFHGLRFPAVNHPQSLERRYLGILNSVLLDLMKNLLKLDPADRYLTEQCLNHPTFQTQRLL

🔍 Best alignment:
SMKIPNIGNVMNKFEILGVVGEGAYGVVLKCRHKETHEIVAIKKFKDSEENEEVKETTLRELKMLRTLKQENIVELKEAFRRRGKLYLVFEYVEKNMLELLEEMPNGVPPEKVKSYIYQLIKAIHWCHKNDIVHRDIKPENLLISHNDVLKLCDFGFARNLSEGNNANYDE-E-VATRWYRSPELLLGAPYGKSVDMWSVGCILGELSDGQPLFPGESEIDQLFTIQKVLGPLPSEQMKLFYSNPRFHGLRFPAVNHPQSLERRYLGILNSVLLDLMKNLLKLDPADRYLTE

/usr/local/lib/python3.11/dist-packages/Bio/PDB/Entity.py:197: BiopythonWarning: The id `(' ', 1, ' ')` is already used for a sibling of this entity. Changing id from `(' ', 2, ' ')` to `(' ', 1, ' ')` might create access inconsistencies to children of the parent entity.
  warnings.warn(
